In [1]:
from lightfm_recomender import LightfmRecomender
import os
%load_ext autoreload
%autoreload 2
new_directory = '/Users/vasevooo/projects/GameBuddy'
os.chdir(new_directory)
import pandas as pd 
import requests
import json
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
from lightfm import LightFM
from scipy.sparse import csr_matrix, save_npz, load_npz
import re
from bs4 import BeautifulSoup

/Users/vasevooo/projects/GameBuddy/steam/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/vasevooo/projects/GameBuddy/steam/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


### обновление предобработанных данных для модели 

In [7]:
games = pd.read_csv('data/game_info_with_scores.csv')
item_dict ={}
games = games[['appid', 'Name']].sort_values('appid').reset_index()
games['appid'] = games['appid'].astype(int)

for i in range(games.shape[0]):
    item_dict[int(games.loc[i,'appid'])] = games.loc[i,'Name']

In [8]:
## сохранение словаря по играм для дальнейшего использования

with open("data/for_models/item_dict.json", "w") as outfile:
    json.dump(item_dict, outfile, indent=4)

### Тест класса LigthFM Recomender

In [2]:
item_dict_path = 'data/for_models/item_dict.json'
gametime_df_path = 'data/final_gametime_df.csv'

lfr = LightfmRecomender(gametime_df_path, item_dict_path)

In [25]:
steam_url = 'https://steamcommunity.com/id/momuss/'

In [31]:
new_user, steamid = lfr.get_user_games(steam_url)

In [32]:
matrix, df = lfr.get_csr_matrix_for_all_users(new_user)

In [33]:
## в приложении поставить 100 эпох. 1 для теста

model = lfr.fit_model(matrix, epochs=1)

In [34]:
lfr.sample_recommendation_user(model, df, steamid, threshold = 0, nrec_items = 10)

User: 76561197994384567
Recommended Items:
1- PAYDAY 2
2- Portal 2
3- The Witcher® 3: Wild Hunt
4- Terraria
5- Half-Life 2
6- Garry's Mod
7- The Elder Scrolls V: Skyrim
8- Dying Light
9- Tomb Raider
10- Don't Starve Together

 Known Likes:
1- Sea of Thieves 2023 Edition
2- Dota Underlords
3- Tropico 6
4- Human: Fall Flat
5- PAYDAY™ The Heist
6- Warhammer® 40,000: Dawn of War® II Chaos Rising
7- Command & Conquer: Red Alert 3
8- Call of Duty®: Modern Warfare® 2 (2009)
9- Call of Duty®: Modern Warfare® 2 (2009)
10- Supreme Commander: Forged Alliance
11- Sid Meier's Civilization® V
12- Warhammer® 40,000: Dawn of War® - Game of the Year Edition
13- Killing Floor
14- Counter-Strike: Global Offensive
15- Left 4 Dead 2
16- Left 4 Dead
17- Counter-Strike: Source
18- Counter-Strike


### cold start

In [23]:
df_cold_start = pd.DataFrame({'game_names': ['Counter-Strike', 'Team Fortress Classic', 
                                             'Day of Defeat', 
                                             'Half-Life: Source', 'Half-Life: Opposing Force'],
                   'rankings': [5, 5, 8, 7, 3]})

In [24]:
df_cold_start.head(2)

,game_names,rankings
0,Counter-Strike,5
1,Team Fortress Classic,5


In [35]:
cold_start_user, steamid_cold = lfr.get_user_games_cold_start(df_cold_start)

In [36]:
matrix_cs, df_cs = lfr.get_csr_matrix_for_all_users(cold_start_user)

In [20]:
## в приложении поставить 100 эпох. 1 для теста
model = lfr.fit_model(matrix_cs, epochs=1)

In [38]:
lfr.sample_recommendation_user(model, df_cs, steamid_cold, threshold = 0, nrec_items = 10)

User: 1234567891010102930
Recommended Items:
1- Counter-Strike: Global Offensive
2- Left 4 Dead 2
3- PAYDAY 2
4- Portal 2
5- The Witcher® 3: Wild Hunt
6- Terraria
7- Half-Life 2
8- Garry's Mod
9- The Elder Scrolls V: Skyrim
10- Dying Light

 Known Likes:
1- Half-Life: Source
2- Day of Defeat
3- Team Fortress Classic
4- Counter-Strike


In [4]:
import pandas as pd 
import requests
import json
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from lightfm import LightFM
from scipy.sparse import csr_matrix
import re
from bs4 import BeautifulSoup


import sqlite3


class LightfmRecomender: 
    
    def __init__(self, gametime_df_path, item_dict_path):

        self.gametime_df = pd.read_csv(gametime_df_path)
        self.scaler = MinMaxScaler(feature_range=(0,10))
        self.gametime_df = self.__gametime_transform(self.gametime_df)
        self.gametime_df.columns = ['steam_id', 'game_id', 'ranking']
        self.item_dict = json.load(open(item_dict_path))
        self.item_dict = {int(k): v for k, v in self.item_dict.items()}
        self.model = LightFM(loss='bpr',
                            random_state=42,
                            learning_rate=0.10,
                            no_components=100,
                            user_alpha=0.000005)
        
    def __get_profile_id(self, url):
        
        userpage = requests.get(url)
        user_page = BeautifulSoup(userpage.content, 'lxml')
        text = str(user_page.find('div', class_="responsive_page_template_content"))
        steamid = int(text[text.find('"steamid"')+11:text.find('"steamid"')+28])
        
        return steamid

    def __gametime_transform(self, df): 
        
        df.iloc[:,:-1] = np.log(df.iloc[:,:-1])
        df.iloc[:,:-1] = df.iloc[:,:-1].replace([np.inf, -np.inf], 0.0)
        df.iloc[:,:-1] = self.scaler.fit_transform(df.iloc[:,:-1])
        transformed_df = pd.melt(df, id_vars='steamid', var_name='game_id', value_name='ranking')
        transformed_df['game_id'] = transformed_df['game_id'].astype(int)
        
        return transformed_df

    def get_user_games(self, url):
        
        steamid = self.__get_profile_id(url)

        url = f'https://api.steampowered.com/IPlayerService/GetOwnedGames/v1/?key=ED07BAAA8209FFE5932318F82B8242A6&format=json&steamid={steamid}'
        response = requests.get(url)
        profile = response.json()

        id_df = pd.DataFrame([steamid], columns=['steam_id'])
        for game in profile['response']['games']:
            appid = game['appid']
            playtime = game['playtime_forever']
            temp_df = pd.DataFrame({
                'steam_id': [steamid],
                appid: [playtime]
            })
            id_df = pd.merge(id_df, temp_df, on='steam_id', how='inner')
       
        transformed_user_profile = pd.melt(id_df, id_vars='steam_id', var_name='game_id', value_name='ranking')
        transformed_user_profile['game_id'] = transformed_user_profile['game_id'].astype(int)
        transformed_user_profile['ranking'] = np.log(transformed_user_profile['ranking'].replace(0, np.nan)).replace(-np.inf, np.nan)
        transformed_user_profile['ranking'] = self.scaler.fit_transform(transformed_user_profile[['ranking']])
        transformed_user_profile['ranking'] = pd.to_numeric(transformed_user_profile['ranking'])
        games_ids_df = self.__get_unique_games_ids()
        merged_df = games_ids_df.merge(transformed_user_profile, on='game_id', how='left')
        merged_df['steam_id'] = steamid
        merged_df.fillna(0, inplace=True)
            
        return merged_df, steamid
    
    def get_user_games_cold_start (self, df): 
        df.columns = ['game_name', 'ranking']
        df['game_id'] = df['game_name'].map({v: k for k, v in self.item_dict.items()})
        df['ranking'] = self.scaler.fit_transform(df[['ranking']])
        df.drop('game_name', axis=1)
        games_ids_df = self.__get_unique_games_ids()
        df = games_ids_df.merge(df, on='game_id', how='left')
        df['steam_id'] = 1234567891010102930
        df.fillna(0, inplace=True)
        
        return df, df['steam_id'][0]

    
    
    def __get_unique_games_ids (self): 
        games_ids = self.gametime_df['game_id'].unique()
        games_ids_df = pd.DataFrame(games_ids, columns=['game_id']).astype(int)

        return games_ids_df
    
    def get_csr_matrix_for_all_users (self, df_user): 
        one_user_interactions = pd.pivot_table(df_user, index='steam_id', columns='game_id', values='ranking')
        all_users_interactions = pd.pivot_table(self.gametime_df, index='steam_id', columns='game_id', values='ranking')
        new_user_added_df = pd.concat([all_users_interactions, one_user_interactions], axis=0)
        new_user_added_df = new_user_added_df.fillna(0)
        new_user_added_df_csr = csr_matrix(new_user_added_df)

        return new_user_added_df_csr, new_user_added_df
    

    def __get_steam_ids(self, df): 
        user_id = list(df.index)
        user_dict = {}
        counter = 0 
        for i in user_id:
            user_dict[i] = counter
            counter += 1
        return user_dict
    
    def fit_model (self, csr_matrix, num_threads = 8, epochs= 100): 
        model = self.model.fit(csr_matrix, num_threads=num_threads, epochs=epochs)

        return model
    
    
    def sample_recommendation_user(self, model, interactions, user_id,
                               threshold = 0,nrec_items = 5, show = True):

        self.user_dict = self.__get_steam_ids(interactions)


        n_users, n_items = interactions.shape
        user_x = self.user_dict[user_id]
        scores = pd.Series(model.predict(user_x,np.arange(n_items), item_features=None))
        scores.index = interactions.columns
        scores = list(pd.Series(scores.sort_values(ascending=False).index))
        
        known_items = list(pd.Series(interactions.loc[user_id,:] \
                                    [interactions.loc[user_id,:] > threshold].index).sort_values(ascending=False))
        
        scores = [x for x in scores if x not in known_items]
        return_score_list = scores[0:nrec_items]
        known_items = [self.item_dict[x] if x in self.item_dict else x for x in known_items]
        scores = [self.item_dict[x] if x in self.item_dict else x for x in return_score_list]
        
        if show == True:
            print ("User: " + str(user_id))
            print("Recommended Items:")
            counter = 1
            for i in scores:
                print(str(counter) + '- ' + str(i))
                counter+=1


            print("\n Known Likes:")
            counter = 1
            for i in known_items:
                print(str(counter) + '- ' + str(i))
                counter+=1
        else:
            return scores


In [8]:
db_path = '/home/nika/GameBuddy/GB.app/database.db'

def view_game_info_table():
    conn = sqlite3.connect(db_path)

    # Чтение данных из таблицы game_info в DataFrame
    query = "SELECT game,rating FROM ratings"
    df = pd.read_sql_query(query, conn)

    conn.close()

    # Возвращение DataFrame
    return df

# Вызов функции для просмотра таблицы game_info в виде DataFrame

df_cold_start = view_game_info_table()

item_dict_path = '/home/nika/GameBuddy/data/for_models/item_dict.json'
gametime_df_path = '/home/nika/GameBuddy/data/final_gametime_df.csv'

lfr = LightfmRecomender(gametime_df_path, item_dict_path)
cold_start_user, steamid_cold = lfr.get_user_games_cold_start(df_cold_start)

matrix_cs, df_cs = lfr.get_csr_matrix_for_all_users(cold_start_user)

model = lfr.fit_model(matrix_cs, epochs=1)

top_10=lfr.sample_recommendation_user(model, df_cs, steamid_cold, threshold = 0, nrec_items = 10,show=False)
def get_games_with_descriptions(top5):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    placeholders = ', '.join(['?'] * len(top5))
    query = f"SELECT Name, description FROM game_info WHERE Name IN ({placeholders})"
    cursor.execute(query, top5)
    rows = cursor.fetchall()
    conn.close()
    game_names = [row[0] for row in rows]
    game_descriptions = [row[1] for row in rows]
    return list(zip(game_names, game_descriptions))

get_games_with_descriptions(top_10)


/home/nika/miniforge3/envs/production/lib/python3.11/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [1]:
import sqlite3
import pandas as pd

In [6]:
# ----------------------------------вот этот кусок вставить для получения списка рекомендаций-------------------------------------------------------------#
from lightfm_recomender import LightfmRecomender

db_path = '/home/nika/GameBuddy/GB.app/database.db'
steam_url = 'https://steamcommunity.com/id/nevernox/'

def lfm_games_with_descriptions(top5):
    conn = sqlite3.connect('/home/nika/GameBuddy/GB.app/database.db')
    cursor = conn.cursor()
    placeholders = ', '.join(['?'] * len(top5))
    query = f"SELECT Name, description FROM game_info WHERE Name IN ({placeholders})"
    cursor.execute(query, top5)
    rows = cursor.fetchall()
    conn.close()
    game_names = [row[0] for row in rows]
    game_descriptions = [row[1] for row in rows]
    return list(zip(game_names, game_descriptions))

def lfm_game_info_table(db_path):
    conn = sqlite3.connect(db_path)
    query = "SELECT game,rating FROM ratings"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df


df_cold_start = lfm_game_info_table(db_path)
item_dict_path = '/home/nika/GameBuddy/data/for_models/item_dict.json'
gametime_df_path = '/home/nika/GameBuddy/data/final_gametime_df.csv'
lfr = LightfmRecomender(gametime_df_path, item_dict_path)

if steam_url is None:
    cold_start_user, steamid_cold = lfr.get_user_games_cold_start(df_cold_start)
    matrix_cs, df_cs = lfr.get_csr_matrix_for_all_users(cold_start_user)
    model = lfr.fit_model(matrix_cs, epochs=1)
    top_10=lfr.sample_recommendation_user(model, df_cs, steamid_cold, threshold = 0, nrec_items = 10,show=False)
else:
    new_user, steamid = lfr.get_user_games(steam_url)
    matrix, df = lfr.get_csr_matrix_for_all_users(new_user)
    model = lfr.fit_model(matrix, epochs=1)
    top_10=lfr.sample_recommendation_user(model, df, steamid, threshold = 0, nrec_items = 10,show=False)

lfm_reccomendations=lfm_games_with_descriptions(top_10)
# ----------------------------------вот этот кусок вставить для получения списка рекомендаций-------------------------------------------------------------#

/home/nika/miniforge3/envs/production/lib/python3.11/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [7]:
lfm_reccomendations

[('Counter-Strike: Global Offensive',
  'Counter-Strike: Global Offensive (CS: GO) expands upon the team-based action gameplay that it pioneered when it was launched 19 years ago. CS: GO features new maps, characters, weapons, and game modes, and delivers updated versions of the classic CS content (de_dust2, etc.).'),
 ('Grand Theft Auto V',
  'Grand Theft Auto V for PC offers players the option to explore the award-winning world of Los Santos and Blaine County in resolutions of up to 4k and beyond, as well as the chance to experience the game running at 60 frames per second.'),
 ("Garry's Mod",
  "Garry's Mod is a physics sandbox. There aren't any predefined aims or goals. We give you the tools and leave you to play."),
 ('PAYDAY 2',
  'PAYDAY 2 is an action-packed, four-player co-op shooter that once again lets gamers don the masks of the original PAYDAY crew - Dallas, Hoxton, Wolf and Chains - as they descend on Washington DC for an epic crime spree.'),
 ('Fallout: New Vegas', 'Welc